# Building JARVIS from Scratch

## Dependencies

In [4]:
import sys
print(sys.executable)


/Users/rohithvs/llm-fix/bin/python


In [5]:
!pip3 install "datasets[audio]"


In [6]:
!pip3 install --upgrade pip
!pip3 install --upgrade transformers sentencepiece "datasets[audio]" accelerate
!pip3 install sounddevice
!pip3 install langchain-huggingface
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121



  Using cached sentencepiece-0.2.0.tar.gz (2.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for sentencepiece (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [120 lines of output]
      /private/var/folders/79/ms14pfqd3s7bwyrgcwcwl1080000gn/T/pip-build-env-9h_dxi1m/overlay/lib/python3.13/site-packages/setuptools/_distutils/dist.py:289: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      /private/var/folders/79/ms14pfqd3s7bwyrgcwcwl1080000gn/T/pip-build-env-9h_dxi1m/overlay/lib/python3.13/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX 

## Whisper

In [7]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip3 install transformers


In [13]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [14]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


In [15]:
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

In [16]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True
)

Device set to use cpu


In [17]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

Generating validation split: 100%|█████████| 1/1 [00:00<00:00,  7.02 examples/s]


In [18]:
result = pipe(sample)
print(result["text"])

/Users/rohithvs/llm-fix/lib/python3.13/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!


## Speech T5

In [22]:
!pip3 install sentencepiece


  Using cached sentencepiece-0.2.0.tar.gz (2.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for sentencepiece (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [133 lines of output]
      /private/var/folders/79/ms14pfqd3s7bwyrgcwcwl1080000gn/T/pip-build-env-e9hke5ov/overlay/lib/python3.13/site-packages/setuptools/_distutils/dist.py:289: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      /private/var/folders/79/ms14pfqd3s7bwyrgcwcwl1080000gn/T/pip-build-env-e9hke5ov/overlay/lib/python3.13/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX 

In [23]:
from transformers import pipeline
from datasets import load_dataset
import soundfile as sf
import torch

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)


ImportError: 
SpeechT5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
speech = synthesiser("Hello, can you explain to me the concept of Deep Learning in a few words?", forward_params={"speaker_embeddings": speaker_embedding})

sf.write("speech.mp3", speech["audio"], samplerate=speech["sampling_rate"])

In [ ]:
from IPython.display import Audio

Audio("speech.mp3", autoplay=True)

## LLAMA

In [ ]:
import torch
from transformers import pipeline
from huggingface_hub import login

login(token='your token here')

model_id = "meta-llama/Llama-3.2-1B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
messages = [
    {"role": "system", "content": "You are my personnal assistant named Jarvis. My name is Julien and you will help me to solve machine learning problems."},
    {"role": "user", "content": "Who are you?"},
    ]

outputs = pipe(
    messages,
    max_new_tokens=256,
    )

print(outputs[0]["generated_text"][-1]["content"])

messages.append(outputs[0]["generated_text"][-1])

In [ ]:
messages

In [ ]:
prompt = {"role": "user", "content": "Who am I Jarvis?"}
messages.append(prompt)

outputs = pipe(messages, max_new_tokens=256)
messages.append(outputs[0]["generated_text"][-1])

print(outputs[0]["generated_text"][-1]["content"])

In [ ]:
prompt = {"role": "user", "content": "Est-ce que tu sais parler français Jarvis? Si oui, réponds moi en Français."}
messages.append(prompt)

outputs = pipe(messages, max_new_tokens=256)
messages.append(outputs[0]["generated_text"][-1])

print(outputs[0]["generated_text"][-1]["content"])

In [ ]:
prompt = {"role": "user", "content": "Quel est le but de la vie Jarvis?"}
messages.append(prompt)

outputs = pipe(messages, max_new_tokens=256)
messages.append(outputs[0]["generated_text"][-1])

print(outputs[0]["generated_text"][-1]["content"])

## Whisper + Speech T5 + Llama

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan, pipeline
from datasets import load_dataset
import torch
import soundfile as sf
from IPython.display import Audio
from huggingface_hub import login

login(token='your token here')

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)


In [ ]:
# whisper to convert input
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

whisper = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)

whisper.to(device)

processor = AutoProcessor.from_pretrained(model_id)

In [ ]:
whisper_pipe = pipeline(
    "automatic-speech-recognition",
    model=whisper,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True
)

In [ ]:
sample = "speech.mp3"

result = whisper_pipe(sample)
print(result['text'])

In [ ]:
# llama for conversation

import torch
from transformers import pipeline
from huggingface_hub import login

login(token='your token here')

model_id = "meta-llama/Llama-3.2-1B-Instruct"

llama_pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
messages = [
    {"role": "system", "content": "You are my personnal assistant named Jarvis. My name is Julien and you will help me to solve machine learning problems."},
    ]

prompt = {"role": "user", "content": "Who am I Jarvis?"}
messages.append(prompt)

In [ ]:
outputs = llama_pipe(
    messages,
    max_new_tokens=256,
    )

print(outputs[0]["generated_text"][-1]["content"])

messages.append(outputs[0]["generated_text"][-1])

In [ ]:
jarvis_answer = outputs[0]["generated_text"][-1]["content"]

speech = synthesiser(jarvis_answer, forward_params={"speaker_embeddings": speaker_embedding})

sf.write("speech.mp3", speech["audio"], samplerate=speech["sampling_rate"])

Audio("speech.mp3", autoplay=True)

In [ ]:
speech = synthesiser("Can you explain me in brief what is Deep Learning? No more than 2 sentences.", forward_params={"speaker_embeddings": speaker_embedding})

sf.write("deepl.mp3", speech["audio"], samplerate=speech["sampling_rate"])

In [ ]:
sample = "deepl.mp3"

result = whisper_pipe(sample)

messages.append({"role": "user", "content": result['text']})

outputs = llama_pipe(
    messages,
    max_new_tokens=256,
    )

print(outputs[0]["generated_text"][-1]["content"])

messages.append(outputs[0]["generated_text"][-1])

In [ ]:
jarvis_answer = outputs[0]["generated_text"][-1]["content"]

speech = synthesiser(jarvis_answer, forward_params={"speaker_embeddings": speaker_embedding})

sf.write("speech.mp3", speech["audio"], samplerate=speech["sampling_rate"])

Audio("speech.mp3", autoplay=True)

## Langchain x HuggingFace

### Llama

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-1B-Instruct",
    task="text-generation",
    max_new_tokens=100,
    do_sample=False,
)

llama = ChatHuggingFace(llm=llm, verbose=True)

messages = [
    ("system", "You are my personnal assistant named Jarvis. My name is Julien and you will help me to solve machine learning problems."),
    ]

In [ ]:
prompt="Who are you Jarvis?"
messages.append(("human", prompt))
answer = llama.invoke(messages)
answer.content

In [ ]:
messages.append(("assistant", answer.content))
messages

In [ ]:
prompt="Can you explain to me what is ML in short?"
messages.append(("human", prompt))
answer = llama.invoke(messages)
answer.content

In [ ]:
messages.append(("assistant", answer.content))
messages

### Complete Chain

In [ ]:
!pip install langchain
!pip install langchain-community

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import soundfile as sf
from huggingface_hub import login
import os

login(token='your token here')

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

whisper_model_id = "openai/whisper-large-v3"
whisper_model = AutoModelForSpeechSeq2Seq.from_pretrained(whisper_model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)
whisper_model.to(device)
whisper_processor = AutoProcessor.from_pretrained(whisper_model_id)

whisper_pipe = pipeline(
    "automatic-speech-recognition",
    model=whisper_model,
    tokenizer=whisper_processor.tokenizer,
    feature_extractor=whisper_processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=False
)

# Llama Initialization
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-1B-Instruct",
    task="text-generation",
    max_new_tokens=100,
    do_sample=False,
)
llama = ChatHuggingFace(llm=llm, verbose=True)

# Speech T5 Initialization
synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# Workflow Function
def conversational_pipeline(audio_path, output_path):
    
    transcription = whisper_pipe(audio_path)["text"]
    print(f"User said: {transcription}")
    
    messages = [
        ("system", "You are my personal assistant named Jarvis."),
        ("human", transcription)
    ]
    
    llama_response = llama.invoke(messages)
    jarvis_answer = llama_response.content
    print(f"Jarvis responded: {jarvis_answer}")

    speech = synthesiser(jarvis_answer, forward_params={"speaker_embeddings": speaker_embedding})
    audio_name = "jarvis_response.mp3"
    output_audio_path = os.path.join(output_path, audio_name)
    sf.write(output_audio_path, speech["audio"], samplerate=speech["sampling_rate"])

    return output_audio_path

audio_file = "../data/deepl.mp3"
output_audio = conversational_pipeline(audio_path=audio_file, output_path='../data')

In [ ]:
from transformers import pipeline

try:
    synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")


In [ ]:
import shutil
import os

# Define the cache path for HuggingFace models
hf_cache_dir = os.path.join(os.path.expanduser("~"), ".cache", "huggingface", "transformers")

# Ensure proper cleanup
model_path = os.path.join(hf_cache_dir, "models--microsoft--speecht5_tts")

if os.path.exists(model_path):
    shutil.rmtree(model_path)
    print("Old model cache removed.")

else:
    print("No local cache found for Microsoft TTS.")

In [ ]:
import os
import transformers

# Check if HF_HOME environment variable is set
hf_home = os.getenv('HF_HOME')
print("HuggingFace home cache directory:", hf_home)

# Check where the HuggingFace cache typically resides
default_cache_path = transformers.utils.default_cache_path
print("Default HuggingFace cache path:", default_cache_path)

In [ ]:
import os

cache_path = os.path.join(os.path.expanduser("~"), ".cache", "huggingface", "hub")

# List contents of the cache path
print(f"Contents of {cache_path}:")
for root, dirs, files in os.walk(cache_path):
    print(root, files)